<a href="https://colab.research.google.com/github/dxda6216/ttron2excel/blob/main/ttron_data_file_to_excel_file_and_data_smoothing_by_moving_average.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Converting Ttron TRACES file (TRACES.nnn) to Excel file
#
# This is a simple script for coverting a Taylortron data file (the Johnson
# Lab, 30 channels, "TRACES.nnn" file) to an Excel file, detrending, smoothing,
# and plotting the data.
# by dxda6216 (Email: dxda6216 AT gmail DOT com)
#
import pandas as pd
import csv
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import MultipleLocator
import io
from google.colab import files

Experiment_number = 'CYxxx' #@param {type:"string"}

#@markdown 1. Input experiment number (avoid spaces and special characters).
#@markdown 2. **Runtime** -> **Restart and run all** (or press **Ctrl+M** and then press **Ctrl+Ente**r)
#@markdown 3. Wait until `Choose Files` or `Browse...` button appears below.
#@markdown 4. Click `Choose Files` or `Browse...` button and select *TRACES.nnn* file in your computer.
#@markdown 5. Wait a while. An Excel file will be saved in "Downloads" folder in your computer.

uploaded = files.upload()
ttronfilename = next(iter(uploaded))

colnames = ["Hours","00","01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29"]
df = pd.read_csv(ttronfilename, header=None, sep ='\t', skiprows=3, skipfooter=1, index_col=False, names=colnames, engine='python')
df_5PMA = df.rolling(window=5, center=True).mean()
df_9PMA = df.rolling(window=9, center=True).mean()
df_33PMA = df.rolling(window=33, center=True).mean()
dtdf = df - df_33PMA
dtdf['Hours'] = df_33PMA['Hours']
dtdf_5PMA = dtdf.rolling(window=5, center=True).mean()
dtdf_9PMA = dtdf.rolling(window=9, center=True).mean()
print(df)

outputexcelfilename = Experiment_number+".xlsx"
with pd.ExcelWriter(outputexcelfilename) as writer:  
	df.to_excel(writer, sheet_name='Raw Data')
	df_5PMA.to_excel(writer, sheet_name='5-point moving average (5PMA)')
	df_9PMA.to_excel(writer, sheet_name='9-point moving average (9PMA)')
	df_33PMA.to_excel(writer, sheet_name='Trend line (33PMA)')
	dtdf.to_excel(writer, sheet_name='Detrended Data')
	dtdf_5PMA.to_excel(writer, sheet_name='Detrended Data 5PMA')
	dtdf_9PMA.to_excel(writer, sheet_name='Detrended Data 9PMA')
	for k in range(0, 30, 1):
		channelnumber = str(k).zfill(2)
		chnum = 'Channel '+channelnumber
		dfx = pd.DataFrame()
		dfx['Hours'] = df['Hours']
		dfx['Raw_data'] = df[channelnumber]
		dfx['5PMA'] = df_5PMA[channelnumber]
		dfx['9PMA'] = df_9PMA[channelnumber]
		dfx['trend_line'] = df_33PMA[channelnumber]
		dfx['detrended_data'] = dtdf[channelnumber]
		dfx['detrended_data_5PMA'] = dtdf_5PMA[channelnumber]
		dfx['detrended_data_9MPA'] = dtdf_9PMA[channelnumber]
		dfx.to_excel(writer, sheet_name=chnum)

plot_output_pdf = Experiment_number + "_data_plots.pdf"
x = df['Hours']
x_scale_min = int(math.floor(min(x)*(1/24)))*24
x_scale_max = int(math.ceil(max(x)*(1/12)))*12+12
xtickslist = list(range(x_scale_min, x_scale_max, 12))

pp = PdfPages(plot_output_pdf)
plt.rcParams.update({'figure.max_open_warning': 0})

fig = plt.figure(figsize=(11, 8.5))
fig.subplots_adjust(hspace=0.15)

plt.rc('font', size=5)
plt.rc('axes', titlesize=4)
plt.rc('axes', labelsize=4)
plt.rc('xtick', labelsize=4)
plt.rc('ytick', labelsize=4)
plt.rc('legend', fontsize=3)
plt.rc('figure', titlesize=5)

for k in range(0, 10, 1):
	for m in range(0, 3, 1):
		channelnumber = str(k + 10 * m).zfill(2)
		subplotnumber = k * 3 + m + 1
		plt.subplot(10, 3, subplotnumber)
		x = df['Hours']
		y = df[channelnumber]
		x_cma = df_33PMA['Hours']
		y_cma = df_33PMA[channelnumber]

		print('Plotting Channel '+channelnumber)		
		chlabel = 'Ch # '+channelnumber
		plt.scatter(x,y,s=0.1,c='blue', label=chlabel)
		plt.plot(x_cma,y_cma,'-r', linewidth=0.5, label='trend line')
		plt.xlim(x_scale_min-6, x_scale_max)
		y_scale_max = int(max(y)*1.100)
		plt.ylim(0, y_scale_max)
		plt.xticks(xtickslist)
		plt.grid(True, linewidth=0.5, color='lightgray', linestyle='--')
		plt.legend(loc='upper right', fontsize=4)
		plt.suptitle(Experiment_number, fontsize=12)

fig.text(0.50, 0.08, 'Time (hours)', horizontalalignment='center', fontsize = 10)
fig.text(0.08, 0.50, 'Bioluminescence', horizontalalignment='center', verticalalignment='center', rotation='vertical', fontsize = 10)
pp.savefig(fig)

fig = plt.figure(figsize=(11, 8.5))
fig.subplots_adjust(hspace=0.15)
for k in range(0, 10, 1):
	for m in range(0, 3, 1):
		channelnumber = str(k + 10 * m).zfill(2)
		subplotnumber = k * 3 + m + 1
		plt.subplot(10, 3, subplotnumber)
		y = df[channelnumber]
		x_dt = dtdf['Hours'] 
		y_dt = dtdf[channelnumber] 
		x_dt_cma = dtdf_5PMA['Hours'] 	
		y_dt_cma = dtdf_5PMA[channelnumber] 
		chlabel = 'Ch # '+channelnumber+' - detrend'
		print('Plotting Channel '+channelnumber+' (detrended)')
		plt.scatter(x_dt,y_dt,s=0.1,c='blue', label=chlabel)
		plt.plot(x_dt_cma,y_dt_cma,'-r', linewidth=0.5)
		plt.xlim(x_scale_min-6, x_scale_max)
		plt.xticks(xtickslist)
		plt.grid(True, linewidth=0.5, color='lightgray', linestyle='--')
		plt.legend(loc='upper right', fontsize=4)
		plt.suptitle(Experiment_number+' - detrended data', fontsize=12)

fig.text(0.50, 0.08, 'Time (hours)', horizontalalignment='center', fontsize = 10)
fig.text(0.08, 0.50, 'Detrended Bioluminescence', horizontalalignment='center', verticalalignment='center', rotation='vertical', fontsize = 10)
pp.savefig(fig)

plt.rc('font', size=10)
plt.rc('axes', titlesize=10)
plt.rc('axes', labelsize=10)
plt.rc('xtick', labelsize=9)
plt.rc('ytick', labelsize=9)
plt.rc('legend', fontsize=6)
plt.rc('figure', titlesize=14)

for k in range(0, 30, 1):
	fig = plt.figure(figsize=(11, 8.5))
	channelnumber = str(k).zfill(2)
	x = df['Hours']
	y = df[channelnumber]
	x_cma = df_33PMA['Hours']
	y_cma = df_33PMA[channelnumber]
	x_dt = dtdf['Hours'] 
	y_dt = dtdf[channelnumber] 
	x_dt_cma = dtdf_5PMA['Hours'] 	
	y_dt_cma = dtdf_5PMA[channelnumber]

	plt.subplot(2, 1, 1)
	print('Plotting Channel '+channelnumber)
	plt.scatter(x,y,s=5.0,c='violet', label='Bioluminescence')
	plt.plot(x_cma,y_cma,'-r', linewidth=1.0, label='Trend line (33-point moving average, centered)')
	plt.xlim(x_scale_min, x_scale_max)
	y_scale_max = int(max(y)*1.100)
	plt.ylim(0, y_scale_max)
	plt.xticks(np.arange(x_scale_min, x_scale_max, 12.0))
	plt.xlabel('Hours', fontsize=12)
	plt.ylabel('Bioluminescence', fontsize=12)
	plt.grid(True, linewidth=0.5, color='lightgray', linestyle='--')
	plt.legend(loc='upper right', fontsize=6)

	plt.subplot(2, 1, 2)
	print('Plotting Channel '+channelnumber+' (detrended)')
	plt.scatter(x_dt,y_dt,s=5.0,c='violet', label='Detrended bioluminescence')
	plt.plot(x_dt_cma,y_dt_cma,'-b', linewidth=1.0, label='5-point moving average, centered')
	plt.xlim(x_scale_min, x_scale_max)
	plt.xticks(np.arange(x_scale_min, x_scale_max, 12.0))
	plt.xlabel('Hours', fontsize=12)
	plt.ylabel('Detrended bioluminescence', fontsize=12)
	plt.grid(True, linewidth=0.5, color='lightgray', linestyle='--')
	plt.legend(loc='upper right', fontsize=6)
	plt.suptitle(Experiment_number+'   Ch # '+channelnumber, fontsize=14)
	pp.savefig(fig)

pp.close()

files.download(outputexcelfilename)
files.download(plot_output_pdf)